# CS108/212 STAT108/212 W26 Course Project

### Team Details

- Teammate 1: Henry Yost
- Teammate 2: Dmitry Sorokin
- Teammate 3: Kyle Chahal
- Teammate 4: Refugio Zepeda

---


# Milestone: Mitigating Bias
For this project milestone, each teammate will implement bias mitigation strategies and assess pre and post bias mitigation performance.

# Installs

In [1]:
# [INSERT CODE HERE to install necessary packages]
import sys
!{sys.executable} -m pip install -r ../requirements.txt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# Imports

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import collections
from pprint import pprint
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo

## Add additional imports needed for your project here.

# Loading dataset
_(same as previous milestone, copy-paste)_

In [3]:
# Load your selected dataset
# fetch dataset 
adult = fetch_ucirepo(id=2) 
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

# variable information 
print(adult.variables)

# Making our data a pandas df
adult_clean = pd.concat([X, y], axis=1)

sensitive_feature_colname = ['age', 'sex', 'race', 'marital-status'] # sensitive feature name
#age, sex, race, (marital status), 

# Make sensitive features-based group labels
group_labels = adult_clean[sensitive_feature_colname]

# Print some stats
print(f"No. of samples: {X.shape[0]}")
print(f"No. of features: {X.shape[1]}")
#print(f"Group Counts: {dict(collections.Counter(group_labels))}")

              name     role         type      demographic  \
0              age  Feature      Integer              Age   
1        workclass  Feature  Categorical           Income   
2           fnlwgt  Feature      Integer             None   
3        education  Feature  Categorical  Education Level   
4    education-num  Feature      Integer  Education Level   
5   marital-status  Feature  Categorical            Other   
6       occupation  Feature  Categorical            Other   
7     relationship  Feature  Categorical            Other   
8             race  Feature  Categorical             Race   
9              sex  Feature       Binary              Sex   
10    capital-gain  Feature      Integer             None   
11    capital-loss  Feature      Integer             None   
12  hours-per-week  Feature      Integer             None   
13  native-country  Feature  Categorical            Other   
14          income   Target       Binary           Income   

                       

# Preparing dataset
_(same as previous milestone, copy-paste)_

In [4]:
# Some subset of following dataset preparation steps may be necessary depending on your dataset,
# 1. Drop unnecessary features
# 2. Handle missing data
# 3. Encode categorical features
# 4. Normalize numerical features
# 5. Encode target (if your task is classification)



#removing unwanted columns
adult_clean = adult_clean.drop(columns = ['education', 'native-country'])
#removing any empty values:
adult_clean = adult_clean.dropna()
#making income binary
    #1 represents over 50 k
mapping = {'>50K': 1, '<=50K': 0}
adult_clean['income'] = adult_clean['income'].map(mapping)
#updating sensitive labels
group_labels = adult_clean[sensitive_feature_colname]
#factorizing non-numeric data:

adult_clean['workclass_num'], unique_labels = pd.factorize(adult_clean['workclass'])
adult_clean['marital-status_num'], unique_labels = pd.factorize(adult_clean['marital-status'])
adult_clean['occupation_num'], unique_labels = pd.factorize(adult_clean['occupation'])
adult_clean['relationship_num'], unique_labels = pd.factorize(adult_clean['relationship'])
adult_clean['race_num'], unique_labels = pd.factorize(adult_clean['race'])
#For sex, male is 1, female is 0
mapping = {'Male': 1, 'Female': 0}
adult_clean['sex'] = adult_clean['sex'].map(mapping)


X = adult_clean[['age', 'workclass_num', 'fnlwgt', 'education-num', 'marital-status_num', 'occupation_num', 'relationship_num', 'race_num', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week']]
y = adult_clean[['income']]



# Note: X and y have been modified before the following lines of code!
print(f"No. of samples AFTER cleaning: {X.shape[0]}")
assert X.shape[0] == y.shape[0] == group_labels.shape[0] ## Ensure that the target and group_labels have been updated if some samples were removed during cleaning.
print(f"No. of features AFTER encoding: {X.shape[1]}")

No. of samples AFTER cleaning: 47876
No. of features AFTER encoding: 12


# Getting training and testing sets

Note: Train-test split is made **ONCE** to obtain the _training set_ and the _testing set_ and every teammate will use the training set to train their baseline model and test the trained model using the testing set. **NEVER** modify the testing set once it has been created.
Therefore, the following code cell does not need to be edited.

_(same as previous milestone, copy-paste)_

In [5]:
X_train, X_test, \
  y_train, y_test, \
    group_labels_train, group_labels_test = train_test_split(X, y, group_labels,
                                                             test_size=0.2, random_state=42)

print(f"No. of training samples: {X_train.shape[0]}")
print(f"No. of testing samples: {X_test.shape[0]}")

# Delete X, y and group_label variables to make sure they are not used later on.
del X
del y
del group_labels

No. of training samples: 38300
No. of testing samples: 9576


# Setting up evaluation metrics
Note: The same evaluation function will be used by all teammates.

_(same as previous milestone, copy-paste)_

In [16]:
from fairlearn.metrics import equalized_odds_difference
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

# double importing just in case
import numpy as np

def evaluate_model(y_test, y_pred, g_labels):
    """
    Evaluate the performance of your trained model on the testing set.
    
    Parameters
    ----------
    y_test : array-like
    The true targets of the testing set.
    y_pred : array-like
    The predicted targets of the testing set.
    g_labels : array-like
    The group labels of the testing set.
    
    Returns
    -------
    results : dict
    A dictionary containing the evaluation results.
    
    Example:
    For classification task, the task-specific performance metrics like {'accuracy': <value>, 'f1_score': <value>, ...}
    and fairness metrics like {'demographic_parity': <value>, 'equalized_odds': <value>, ...}.
    """
    results = {}

    # force them to be arrays just in case, so we don't have an error
    y_test = np.asarray(y_test).ravel()
    y_pred = np.asarray(y_pred).ravel()
    g_labels = np.asarray(g_labels).ravel()
    
    # Note: These metrics will be calculated for - 1. the full testing set, 2. individual groups.
    # Task-specific performance metrics

    global_accuracy = accuracy_score(y_test, y_pred)
    print(f"Global Accuracy score of: {global_accuracy}")
    results["accuracy_overall"] = global_accuracy

    global_f1 = f1_score(y_test, y_pred)
    print(f"Global f1 score of: {global_f1}")
    results["f1_overall"] = global_f1

    results["accuracy_by_group"] = {}
    results["f1_by_group"] = {}

    for g in np.unique(g_labels):
        mask = (g_labels == g)
        y_test_g = y_test[mask]
        y_pred_g = y_pred[mask]

        results["accuracy_by_group"][g] = accuracy_score(y_test_g, y_pred_g)
        results["f1_by_group"][g] = f1_score(y_test_g, y_pred_g, pos_label=1)
    
    # Fairness metric:
    # The fairness metric we will be using is equalied odds, because: Equalized odds requires the TPR and FPR are equal accross all protected groups.

    eo_diff = equalized_odds_difference(y_test, y_pred, sensitive_features=g_labels, method='between_groups')
    print(f"Equalized Odds Difference: {eo_diff}")
    results['eo_diff'] = eo_diff
    
    return results

# Training baseline models (INDIVIDUAL CONTRIBUTION)
_(minor modifications from previous milestone)_

In [7]:
## A place to save all teammates's baseline results
all_baseline_results = [] ## DO NOT EDIT

## Teammate 1

In [8]:
# Select a model and train it on the training set
# Deciding to use a KNN model

#normalizing training data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#finding optimal number of neighbors from 1 to 10, arbitrarily chosen, to limit processing time.
optimal_n = 0
optimal_n_acc = 0

for i in range(1, 11):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)
    accuracy = knn.score(X_test, y_test)

    if(0 == optimal_n):
        optimal_n = i
        optimal_n_acc = accuracy
    elif(accuracy > optimal_n_acc):
        optimal_n = i
        optimal_n_acc = accuracy

#Making the actual model:
KNN_model = KNeighborsClassifier(n_neighbors=optimal_n)
KNN_model.fit(X_train, y_train)
y_pred = KNN_model.predict(X_test)


# Make predictions on the testing set and store them in y_pred
y_pred = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
# results = evaluate_model(y_test, y_pred, group_labels_test)

# # Save your results to all_baseline_results
# results['teammate'] = 'Teammate 1'
# results['experiment_type'] = 'baseline'
# results['predictor_model'] = ... #[INSERT MODEL NAME HERE]
# results['mitigation_strategy'] = 'NONE' ## DO NOT EDIT: This is pre-mitigation baseline
# all_baseline_results.append(results)

# pprint(results)

ValueError: Input y contains NaN.

## Teammate 2

In [ ]:
# Select a model and train it on the training set
# [INSERT YOUR CODE HERE]

# Make predictions on the testing set and store them in y_pred
y_pred = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results = evaluate_model(y_test, y_pred, group_labels_test)

# Save your results to all_baseline_results
results['teammate'] = 'Teammate 2'
results['experiment_type'] = 'baseline'
results['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results['mitigation_strategy'] = 'NONE' ## DO NOT EDIT: This is pre-mitigation baseline
all_baseline_results.append(results)

pprint(results)

## Teammate 3

In [ ]:
# Select a model and train it on the training set
# [INSERT YOUR CODE HERE]

# Make predictions on the testing set and store them in y_pred
y_pred = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results = evaluate_model(y_test, y_pred, group_labels_test)

# Save your results to all_baseline_results
results['teammate'] = 'Teammate 3'
results['experiment_type'] = 'baseline'
results['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results['mitigation_strategy'] = 'NONE' ## DO NOT EDIT: This is pre-mitigation baseline
all_baseline_results.append(results)

pprint(results)

## Teammate 4

In [ ]:
# Select a model and train it on the training set
# [INSERT YOUR CODE HERE]

# Make predictions on the testing set and store them in y_pred
y_pred = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results = evaluate_model(y_test, y_pred, group_labels_test)

# Save your results to all_baseline_results
results['teammate'] = 'Teammate 4'
results['experiment_type'] = 'baseline'
results['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results['mitigation_strategy'] = 'NONE' ## DO NOT EDIT: This is pre-mitigation baseline
all_baseline_results.append(results)

pprint(results)

# Mitigating Bias (INDIVIDUAL CONTRIBUTION)

_(new in this milestone)_


In [ ]:
## A place to save all teammates' post-mitigation results
all_mitigated_results = [] ## DO NOT EDIT

## Teammate 1

In [ ]:
# Implement your bias mitigation strategy
## If you chose preprocessing, you will train a new version of your predictor model with new/modified inputs.
## If you chose inprocessing, you will train a new version of your predictor with modified learning objective (loss function).
## If you chose postprocessing, you will implement strategies to modify the predictions (y_pred) of the trained baseline predictor model from the previous milestone without training any new version of the predictor model.

# [INSERT CODE HERE]

# Make predictions on the testing set and store them in y_pred_mitigate
y_pred_mitigated = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results_mitigated = evaluate_model(y_test, y_pred_mitigated, group_labels_test)

# Save your results to all_mitigated_results
results_mitigated['teammate'] = 'Teammate 1'
results_mitigated['experiment_type'] = 'post-mitigation'
results_mitigated['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results_mitigated['mitigation_strategy'] = ... #[INSERT STRATEGY TYPE HERE: 'preprocessing', 'inprocessing', 'postprocessing']
all_mitigated_results.append(results_mitigated)

pprint(results_mitigated)

### Teammate 1's Conclusions
[Briefly describe findings and conclusions here. Compare post-mitigation results with baseline results for your model. What is the % improvement in performance post-mitigation?  ]

## Teammate 2

In [ ]:
# Implement your bias mitigation strategy
## If you chose preprocessing, you will train a new version of your predictor model with new/modified inputs.
## If you chose inprocessing, you will train a new version of your predictor with modified learning objective (loss function).
## If you chose postprocessing, you will implement strategies to modify the predictions (y_pred) of the trained baseline predictor model from the previous milestone without training any new version of the predictor model.

# [INSERT CODE HERE]

# Make predictions on the testing set and store them in y_pred_mitigate
y_pred_mitigated = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results_mitigated = evaluate_model(y_test, y_pred_mitigated, group_labels_test)

# Save your results to all_mitigated_results
results_mitigated['teammate'] = 'Teammate 2'
results_mitigated['experiment_type'] = 'post-mitigation'
results_mitigated['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results_mitigated['mitigation_strategy'] = ... #[INSERT STRATEGY TYPE HERE: 'preprocessing', 'inprocessing', 'postprocessing']
all_mitigated_results.append(results_mitigated)

print(results_mitigated)

### Teammate 2's Conclusions
[Briefly describe findings and conclusions here. Compare post-mitigation results with baseline results for your model. What is the % improvement in performance post-mitigation?]


## Teammate 3

In [ ]:
# Implement your bias mitigation strategy
## If you chose preprocessing, you will train a new version of your predictor model with new/modified inputs.
## If you chose inprocessing, you will train a new version of your predictor with modified learning objective (loss function).
## If you chose postprocessing, you will implement strategies to modify the predictions (y_pred) of the trained baseline predictor model from the previous milestone without training any new version of the predictor model.

# [INSERT CODE HERE]

# Make predictions on the testing set and store them in y_pred_mitigate
y_pred_mitigated = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results_mitigated = evaluate_model(y_test, y_pred_mitigated, group_labels_test)

# Save your results to all_mitigated_results
results_mitigated['teammate'] = 'Teammate 3'
results_mitigated['experiment_type'] = 'post-mitigation'
results_mitigated['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results_mitigated['mitigation_strategy'] = ... #[INSERT STRATEGY TYPE HERE: 'preprocessing', 'inprocessing', 'postprocessing']
all_mitigated_results.append(results_mitigated)

print(results_mitigated)

### Teammate 3's Conclusions
[Briefly describe findings and conclusions here. Compare post-mitigation results with baseline results for your model. What is the % improvement in performance post-mitigation?]


## Teammate 4

In [ ]:
# Implement your bias mitigation strategy
## If you chose preprocessing, you will train a new version of your predictor model with new/modified inputs.
## If you chose inprocessing, you will train a new version of your predictor with modified learning objective (loss function).
## If you chose postprocessing, you will implement strategies to modify the predictions (y_pred) of the trained baseline predictor model from the previous milestone without training any new version of the predictor model.

# [INSERT CODE HERE]

# Make predictions on the testing set and store them in y_pred_mitigate
y_pred_mitigated = ... # [INSERT CODE HERE]

# Evaluate testing set predictions using evaluate_model()
results_mitigated = evaluate_model(y_test, y_pred_mitigated, group_labels_test)

# Save your results to all_mitigated_results
results_mitigated['teammate'] = 'Teammate 4'
results_mitigated['experiment_type'] = 'post-mitigation'
results_mitigated['predictor_model'] = ... #[INSERT MODEL NAME HERE]
results_mitigated['mitigation_strategy'] = ... #[INSERT STRATEGY TYPE HERE: 'preprocessing', 'inprocessing', 'postprocessing']
all_mitigated_results.append(results_mitigated)

print(results_mitigated)

### Teammate 4's Conclusions
[Briefly describe findings and conclusions here. Compare post-mitigation results with baseline results for your model. What is the % improvement in performance post-mitigation?]


# Conclusions
_(new in this milestone)_


In [ ]:
# Collect all the results in one table.
overall_results = pd.concat([pd.DataFrame(all_baseline_results), pd.DataFrame(all_mitigated_results)])
overall_results ## Note: The table displayed below in this starter notebook is for your reference, your team's table will be slightly different (e.g. different metrics, no.of sensitive attribute-based groups, actual values, etc.) upon successful completion of this notebook.

[Briefly describe overall findings and conclusions here. Which mitigation strategy resulted in most improvement? Which resulted in the least improvement? Visualize the results with some informative plots. (Hint: Use the `overall_results` table).]

# References

[List the references you used to complete this milestone here.]
- Teammate 1:
- Teammate 2:
- Teammate 3:
- Teammate 4:

# Disclosures

[Disclose use of generative AI and similar tools here.]
- Teammate 1:
- Teammate 2:
- Teammate 3:
- Teammate 4: